In [1]:
import pandas as pd
import os

In [2]:
def create_df():
    csv_files = [f for f in os.listdir('inputs') if f.endswith(".csv")]
    csv_file = str(csv_files[0])
    df = pd.read_csv(f'inputs/{csv_file}')
    payments = df.shape[0]+1
    print(f'{payments} payments were found in this file')
    return df
df = create_df()

142 payments were found in this file


In [3]:
df = df.groupby(['resources.description'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
df

,resources.description,gross_amount,gocardless_fees,app_fees,net_amount
0,"Activities (Camps, Sleepovers etc): Beavers (S...",30,0.46,0.59,28.95
1,"Activities (Camps, Sleepovers etc): Cubs (Chri...",25,0.41,0.49,24.10
2,"Activities (Camps, Sleepovers etc): Cubs (Summ...",90,1.38,1.77,86.85
3,"Activities (Camps, Sleepovers etc): Scouts (20...",696,8.52,13.56,673.92
4,"Activities (Camps, Sleepovers etc): Scouts (Su...",330,5.06,6.49,318.45
5,Monthly Subscriptions: Beavers (2023 November),42,0.58,0.46,22.96
6,Monthly Subscriptions: Beavers (2023 October),12,0.29,0.23,11.48
7,Monthly Subscriptions: Beavers (December 23),12,0.29,0.23,11.48
8,Monthly Subscriptions: Beavers (January 24),240,5.80,4.60,229.60
9,Monthly Subscriptions: Cubs (February 2024),348,8.41,6.67,332.92


In [4]:
sections = ['Squirrels', 'Beavers', 'Cubs', 'Scouts']
payment_schedules = ['Activities', 'Monthly Subscriptions']

result_subsets = []  # List to store subsets

for section in sections:
    subset_section = df[df['resources.description'].str.contains(section, case=False)].copy()
    subset_section['Section'] = section

    for schedule in payment_schedules:
        subset_schedule = subset_section[subset_section['resources.description'].str.contains(schedule, case=False)].copy()
        subset_schedule['Payment Schedule'] = schedule
        subset_schedule['Section Payment Schedule'] = section + ": " + schedule

        result_subsets.append(subset_schedule)

# Concatenate all subsets into a new DataFrame
df = pd.concat(result_subsets, ignore_index=True)
df = df.groupby(['Section Payment Schedule'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
df = df.sort_values(by=['Section Payment Schedule'])

# Display the result DataFrame
display(df)

,Section Payment Schedule,gross_amount,gocardless_fees,app_fees,net_amount
0,Beavers: Activities,30,0.46,0.59,28.95
1,Beavers: Monthly Subscriptions,306,6.96,5.52,275.52
2,Cubs: Activities,115,1.79,2.26,110.95
3,Cubs: Monthly Subscriptions,348,8.41,6.67,332.92
4,Scouts: Activities,1026,13.58,20.05,992.37
5,Scouts: Monthly Subscriptions,540,13.05,10.35,516.60
6,Squirrels: Monthly Subscriptions,180,4.35,3.45,172.20
